In [ ]:
%cd ~/projects/wind/

In [ ]:
import requests
import polars as pl
import json
from wind.preprocess.prepare_local_data import get_local_windpower
import plotly.express as px


## Stations

In [ ]:
windparks_nve = pl.read_csv("data/windparks_nve.csv", separator=";", decimal_comma=True)
power_nve = get_local_windpower("data/windpower2002-2024_utcplus1.csv")

# Check if any wind parks in the production data are missing from the station overview from NVE
windparks_nve_ids = windparks_nve.select("KraftverkID").to_series().implode()
power_nve.select(pl.col("windpark_nve_id").unique()).filter(
    ~pl.col("windpark_nve_id").is_in(windparks_nve_ids)
).collect()

In [ ]:
stations = requests.get(
    "https://ummapi.nordpoolgroup.com/infrastructure/stations"
).json()
assets = requests.get("https://ummapi.nordpoolgroup.com/infrastructure/assets").json()
windpark_nordpool = (
    pl.concat([pl.json_normalize(assets), pl.json_normalize(assets)])
    .unique()
    .select(
        pl.col("name").alias("windpark_name_nordpool"),
        pl.col("code").alias("eic_nordpool"),
    )
)
windpark_nordpool

In [ ]:
windparks_statnet = (
    pl.read_csv("data/windparks_bidzone.csv")
    .select(
        "bidding_area",
        windpark_name_statnet="substation_name",
        eic_code=pl.col("eic_code").replace(
            {"50WI00000001984G": "50WP00000002343W"}
        ),  # Replace EIC code for Mehuken to match with NordPool
    )
    .unique()
)
windparks_statnet

In [ ]:
# Check if any stations from the NVE overview are missing from station lookup from NordPool
lookup_names = windpark_nordpool.select("name").to_series().implode()
windparks_nve.filter(~pl.col("Kraftverknavn").is_in(lookup_names))

In [ ]:
# Check if any stations from the Statnet overview are missing from station lookup from NordPool
lookup_codes = windpark_nordpool.select("code").to_series().implode()
windparks_statnet.filter(~pl.col("eic_code").is_in(lookup_codes))

### Consolidation Strategy
We want to be able to join the wind park overview we have with messages from NordPool.
To do this we have to have an EIC code for each station. 
The priority is to ensure that all stations in the local power production from NVE are correct and have a match as these should sum up to the area production.
The station overview from Statnet has EIC codes which all, except (Mehuken, 50WI00000001984G), have a match with codes from NordPool.
Names between the NVE and Statnet overviews are not consistent so we will have to create a lookup table to join these.   

In [ ]:
windparks_statnet_nordpool = windparks_statnet.join(
    windpark_nordpool, left_on="eic_code", right_on="eic_nordpool", how="left"
)

windparks_match = (
    windparks_nve.select(
        pl.col("Kraftverknavn").alias("windpark_name_nve"),
        pl.col("KraftverkID").alias("windpark_id_nve"),
    )
    .join(
        windparks_statnet_nordpool,
        left_on="windpark_name_nve",
        right_on="windpark_name_statnet",
        how="full",
        coalesce=False,
    )
    .sort("windpark_name_nve", "windpark_name_nordpool")
)
windparks_match.write_csv("data/windparks_lookup.csv")

In [ ]:
windparks_no_match = (
    windparks_nve.select(
        pl.col("Kraftverknavn").alias("windpark_name_nve"),
        pl.col("KraftverkID").alias("windpark_id_nve"),
    )
    .join(
        windparks_statnet_nordpool,
        left_on="windpark_name_nve",
        right_on="windpark_name_statnet",
        how="full",
        coalesce=False,
    )
    .filter(
        pl.col("windpark_name_nve").is_null()
        | pl.col("windpark_name_statnet").is_null()
    )
    .sort("windpark_name_nve", "windpark_name_nordpool")
)
windparks_no_match.write_csv("data/windparks_no_match.csv")

In [ ]:
windparks_match = pl.read_csv("data/windparks_lookup.csv")

In [ ]:
res = requests.get("https://api.nve.no/web/WindPowerplant/GetWindPowerPlants")
windparks_api = pl.json_normalize(res.json())
windparks_api.head(5)

In [ ]:
power_nve.group_by("windpark_nve_id").agg(
    max_output=pl.col("local_power").max()
).collect()

In [ ]:
windparks_api.explode("Turbiner").group_by(
    "Navn", "VindkraftAnleggId", "InstallertEffekt_MW"
).agg(
    sum_effect=(
        pl.col("Turbiner").struct.field("TurbinStorrelse_kW")
        * pl.col("Turbiner").struct.field("AntallTurbiner")
    ).sum()
    / 1000.0
).join(
    power_nve.group_by("windpark_nve_id")
    .agg(max_output=pl.col("local_power").max())
    .collect(),
    left_on="VindkraftAnleggId",
    right_on="windpark_nve_id",
    how="left",
)

In [ ]:
windparks_match.select("windpark_name_nve", "windpark_id_nve").unique().join(
    windparks_api, left_on="windpark_id_nve", right_on="VindkraftAnleggId", how="left"
).filter(pl.col("GjsnittNavhoeyde").is_null())

## Geo-Features

In [ ]:
import geopandas as gpd
import xarray as xr

In [ ]:
windparks_geo = gpd.read_file("data/NVEData.gdb/")
# windparks_geo = windparks_geo[~windparks_geo["idriftDato"].isnull()]
windpark_names = (
    windparks_match.select("windpark_nve_id", "windpark_nve")
    .unique()
    .with_columns(
        pl.col("windpark_nve").replace(
            {
                "Nye Sandøy": "Sandøy",
                "Valsneset vindkraftverk": "Valsneset",
                "Raggovidda 2": "Raggovidda",
            }
        )
    )
    .to_pandas()
)
# windparks_geo = windpark_names.merge(
#     windparks_geo, how="inner", left_on="windpark_nve", right_on="saksTittel"
# )
windparks_geo = windparks_geo.merge(
    windpark_names, how="inner", left_on="saksTittel", right_on="windpark_nve"
)
windparks_geo.columns

In [ ]:
windparks_geo["has_start_date"] = ~windparks_geo["idriftDato"].isnull()
windparks_geo = windparks_geo.sort_values(
    ["windpark_nve_id", "has_start_date"]
).drop_duplicates("windpark_nve_id", keep="first")
out[out["saksTittel"] == "Lindesnes"].iloc[0]


In [ ]:
coords = windparks_geo.geometry.get_coordinates()
xi = xr.DataArray(
    coords.x.values, dims="point", coords={"point": windparks_geo.index}, name="x"
)
yi = xr.DataArray(
    coords.y.values, dims="point", coords={"point": windparks_geo.index}, name="y"
)

In [ ]:
topo = xr.open_dataarray("data/topography.tif").squeeze()
station_height = topo.interp(
    x=xi,
    y=yi,
)

In [ ]:
rix = xr.open_dataarray(
    "data/Vindressurs/Vindressurs_Terrengkompleksitet.tif"
).squeeze()
coords = windparks_geo.geometry.to_crs(
    rix.spatial_ref.attrs["crs_wkt"]
).get_coordinates()
xi = xr.DataArray(
    coords.x.values, dims="point", coords={"point": windparks_geo.index}, name="x"
)
yi = xr.DataArray(
    coords.y.values, dims="point", coords={"point": windparks_geo.index}, name="y"
)
ruggedness = rix.interp(
    x=xi,
    y=yi,
)
# ruggedness = rix.sel(x=xi, y=yi, method="nearest")
ruggedness.values

In [ ]:
windparks_geo.iloc[21]

In [ ]:
rix

## NordPool

In [ ]:
areas = [
    {"name": "NO1", "code": "10YNO-1--------2"},
    {"name": "NO2", "code": "10YNO-2--------T"},
    {"name": "NO3", "code": "10YNO-3--------J"},
    {"name": "NO4", "code": "10YNO-4--------9"},
]
url = "https://ummapi.nordpoolgroup.com/messages"

messages = []
skip = 0
while True:
    res = requests.get(
        url,
        params={
            "limit": 2000,
            # "messageTypes": "TransmissionUnavailability",
            # "messageTypes": "ProductionUnavailability",
            "messageTypes": "MarketInformation",
            "areas": [a["code"] for a in areas],
            # "areas": areas[4]["code"],
            # "fuelTypes": 19,
            # "publicationStartDate": "2020-01-01T00:00:00",
            "skip": skip,
        },
    )
    if res.status_code != 200:
        print(res.status_code)
        break

    content = res.json()
    if len(content["items"]) == 0:
        break
    messages.extend(content["items"])
    skip += len(content["items"])
    print(
        f"Retrieved: {len(content['items'])} ---- Progress: {skip}/{content['total']}"
    )
    if skip >= content["total"]:
        break

In [ ]:
unit_fields = [
    "name",
    "eic",
    "fuelType",
    "areaEic",
    "areaName",
    "installedCapacity",
    "timePeriods",
]
time_period_fields = [
    "unavailableCapacity",
    "availableCapacity",
    "eventStart",
    "eventStop",
]
production = (
    pl.json_normalize(messages, infer_schema_length=1000)
    .filter(pl.col("messageType") == 1, pl.col("generationUnits").is_null())
    # .filter(pl.col("generationUnits").is_not_null())
    .explode("productionUnits")
    .select(
        "messageId",
        "messageType",
        (pl.col("unavailabilityType") == 1).alias("unplannedEvent"),
        "reasonCode",
        "unavailabilityReason",
        "remarks",
        "publicationDate",
        "publisherId",
        "publisherName",
        *(pl.col("productionUnits").struct.field(u).alias(u) for u in unit_fields),
    )
    # .filter((pl.col("fuelType") == 19) | (pl.col("fuelType").is_null()))
    .explode("timePeriods")
    .with_columns(
        pl.col("timePeriods").struct.field(u).alias(u) for u in time_period_fields
    )
)
production

In [ ]:
messages

In [ ]:
market = (
    pl.json_normalize(messages, infer_schema_length=1000)
    .filter(pl.col("messageType") == 5)
    .explode("areas")
    .select(
        "messageId",
        "eventStart",
        "eventStop",
        "remarks",
        "publicationDate",
        "publisherId",
        "publisherName",
        "assets",
        pl.col("areas").struct.field("name").alias("areaName"),
        pl.col("areas").struct.field("code").alias("areaEic"),
        # "marketParticipants",
    )
    .explode("assets")
    .with_columns(
        pl.col("assets").struct.field("name").alias("name"),
        pl.col("assets").struct.field("code").alias("eic"),
    )
    .drop("assets")
)
market

In [ ]:
windparks_match = pl.read_csv("data/windparks_lookup.csv")
market.filter(
    pl.col("eic").is_not_null(), pl.col("areaName").is_in(["NO1", "NO2", "NO3", "NO4"])
).join(windparks_match, how="cross").filter(
    (pl.col("eic") == pl.col("eic_code"))
    | (pl.col("name") == pl.col("windpark_name_nve"))
    | (pl.col("name") == pl.col("windpark_name_statnet"))
    | (pl.col("name") == pl.col("windpark_name_nordpool"))
)

In [ ]:
from datetime import datetime

transmission = (
    pl.json_normalize(messages, infer_schema_length=1000)
    .filter(pl.col("messageType") == 3)
    .explode("transmissionUnits")
    .select(
        "messageId",
        "messageType",
        pl.col("eventStart").cast(pl.Datetime).alias("transmissionEventStart"),
        pl.col("eventStop").cast(pl.Datetime).alias("transmissionEventStop"),
        (pl.col("unavailabilityType") == 1).alias("unplannedEvent"),
        "remarks",
        pl.col("publicationDate").cast(pl.Datetime),
        "publisherId",
        "publisherName",
        "transmissionUnits",
        pl.col("transmissionUnits").struct.field("inAreaName").alias("inAreaName"),
        pl.col("transmissionUnits").struct.field("outAreaName").alias("outAreaName"),
        pl.col("transmissionUnits")
        .struct.field("installedCapacity")
        .alias("installedCapacity"),
        pl.col("transmissionUnits").struct.field("timePeriods").alias("timePeriods"),
    )
    .filter(
        pl.col("inAreaName").is_in(["NO1", "NO2", "NO3", "NO4"])
        | pl.col("outAreaName").is_in(["NO1", "NO2", "NO3", "NO4"])
    )
    .explode("timePeriods")
    .with_columns(
        pl.col("timePeriods")
        .struct.field("unavailableCapacity")
        .alias("unavailableCapacity"),
        pl.col("timePeriods")
        .struct.field("availableCapacity")
        .alias("availableCapacity"),
        pl.col("timePeriods")
        .struct.field("eventStart")
        .cast(pl.Datetime)
        .alias("eventStart"),
        pl.col("timePeriods")
        .struct.field("eventStop")
        .cast(pl.Datetime)
        .alias("eventStop"),
    )
    .filter(pl.col("eventStop") > datetime(2020, 1, 1))
    .select(
        # pl.col("transmissionEventStart"),
        # pl.col("transmissionEventStop"),
        "outAreaName",
        "inAreaName",
        "unplannedEvent",
        "installedCapacity",
        "unavailableCapacity",
        "availableCapacity",
        (pl.col("unavailableCapacity") / pl.col("installedCapacity")).alias(
            "missing_transmission"
        ),
        "publicationDate",
        "eventStart",
        "eventStop",
    )
)
transmission

In [ ]:
transmission.filter(pl.col("outAreaName").is_in(["NO1", "NO2", "NO3", "NO4"]))

In [ ]:
data = pl.scan_parquet("data/windpower_area_dataset.parquet")
data.select("time_ref", "time", "bidding_area").unique().collect()

In [ ]:
unavailable_transmission = (
    data.filter(pl.col("bidding_area") == "ELSPOT NO1")
    .select("time_ref", "time", "bidding_area")
    .join(
        transmission.lazy(),
        left_on=pl.col("bidding_area").str.tail(3),
        right_on="outAreaName",
        how="left",
    )
    .filter(
        pl.col("time_ref") > pl.col("publicationDate"),
        pl.col("time") >= pl.col("eventStart"),
        pl.col("time") <= pl.col("eventStop"),
    )
    .group_by("time_ref", "time", "bidding_area")
    .agg(
        pl.col("unplannedEvent").max(),
        pl.col("unavailableCapacity").sum(),
        pl.col("installedCapacity").sum(),
    )
    .with_columns(
        unavailable_transmission=pl.col("unavailableCapacity")
        / pl.col("installedCapacity")
    )
)

unavailable_transmission = data.join(
    unavailable_transmission, on=["time_ref", "time", "bidding_area"], how="left"
).select(
    "time_ref",
    "time",
    "bidding_area",
    pl.col("unplannedEvent").fill_null(False),
    pl.col("unavailable_transmission").fill_null(0),
)

In [ ]:
import plotly.express as px
from datetime import timedelta

px.line(
    unavailable_transmission.filter(
        pl.col("time") >= pl.col("time_ref").dt.date() + timedelta(days=1),
        pl.col("time") < pl.col("time_ref").dt.date() + timedelta(days=2),
    )
    .sort("time")
    .collect(),
    "time",
    "unavailable_transmission",
)